Elasticsearch
# Indexación de documentos

## Indexación de documentos
 - Gestión de los índices 
 - Gestión de los esquemas (mappings) 
 - Tipos de datos
 - Análisis de texto

## Gestión de los índices
 - ES gestiona automaticamente la creacion de indices y tipos de documentos al crear un índice
     - Sin embargo, no siempre es el comportamiento deseado
 - Podemos gestionar índices con la API de indices
    - operaciones CRUD sobre un índice: GET, POST, PUT, DELETE 
 - Indexacion es un proceso generalmente batch => no es posible cambiar muchas opciones despues de haber creado el índice => requiere reindexar 
 - ES tiene la opcion de indexar de forma incremental, incluso en (Near) Real Time 
 

## Obteniendo información de un índice

In [6]:
import requests

# El indice no existe
r = requests.get('http://localhost:9200/myindex?pretty')
print r.text

{
  "error" : {
    "root_cause" : [ {
      "type" : "index_not_found_exception",
      "reason" : "no such index",
      "resource.type" : "index_or_alias",
      "resource.id" : "myindex",
      "index" : "myindex"
    } ],
    "type" : "index_not_found_exception",
    "reason" : "no such index",
    "resource.type" : "index_or_alias",
    "resource.id" : "myindex",
    "index" : "myindex"
  },
  "status" : 404
}



## Creando un índice
 - al crear podemos definir parámetros de configuración: 
    - Replicacion y distribución del índice
    - Analizadores por defecto
    - Medida de relevancia 
    - etc.

## Creando un índice

In [24]:
index_options = """
{
    "settings" : {
        "index" : {
            "number_of_shards" : 3,
            "number_of_replicas" : 2
        }
    }
}
"""

# Creamos un indice 
r = requests.put('http://localhost:9200/tvseries', data = index_options)
r.json()

{u'acknowledged': True}

In [22]:
r = requests.get('http://localhost:9200/tvseries?pretty')
print r.text

{
  "tvseries" : {
    "aliases" : { },
    "mappings" : { },
    "settings" : {
      "index" : {
        "creation_date" : "1455361568534",
        "number_of_shards" : "3",
        "number_of_replicas" : "2",
        "uuid" : "A7BeQL6BQW-eVEVm8UaN2Q",
        "version" : {
          "created" : "2010099"
        }
      }
    },
    "warmers" : { }
  }
}



In [13]:
r = requests.get('http://localhost:9200/tvseries/_aliases?pretty')
print r.text

{
  "tvseries" : {
    "aliases" : { }
  }
}



## Borrando el índice

In [23]:
# Borramos un indice 
r = requests.delete('http://localhost:9200/tvseries')
r.json()

{u'acknowledged': True}

## Cerrando y abriendo índices 
  - Un indice cerrado
    - no se puede consultar 
    - no requiere espacio en memoria
    - permanece usando disco

In [20]:
r = requests.post('http://localhost:9200/tvseries/_close')
print r.text

{"acknowledged":true}


In [25]:
r = requests.post('http://localhost:9200/tvseries/_open')
print r.text

{"acknowledged":true}


## Otras operaciones de administración sobre el índice

Estas operaciones se realizan de forma automática, pero se pueden forzar de forma manual 

  - **Optimizar** - Reduce el número de segmentos de índice de Lucene. A medida que se indexan documentos aumenta el número de segmentos que Lucene usa. Podemos forzar a que cree segmentos los más grandes posibles. 
  - **Refrescar** - Actualiza las últimas operaciones del índice
  - **Flush**  - Libera memoria y actualiza el log transaccional 
  - **Limpiar cache** - vacia la cache del índice. Por razones de rendimiento cada nodo mantiene ciertas partes del índice. 
  
Se invocan como las operaciones de cerrar y abrir:  
  <pre> Ejemplo: POST &lt;index&gt;/_flush </pre>

## Tipos de documentos

En una BBDD relacional se definen las entidades y el tipo de datos en el esquema físico => CREATE TABLE

En un sistema de búsqueda, los tipos de documentos se definen mediante un mapeo (*mappings*)
  - ES no requiere definir un esquema, puede inferirlo => *dynamic mapping*
  - *dynamic mapping* es intereante durante el desarrollo
  - Sin embargo, generalmente se requiere más control
  

##  Mapeos (*Mappings*) 
  - Un mapeo permite definir los tipos de cada uno de los campos de un documentos y como se almacenan
  - Cada mapeo generalmente está asociado a un índice. 
  - La API de mappings permite: 
     - Gestionar los mapeos - operaciones CRUD 
     - Añadir o borrar campos a cada uno de los mapeos

## Obteniendo el mapeo de un tipo
  - El índice *megacorps* y el tipo *employee* se crearon por defecto al añadir un documento 

In [27]:
employee = """
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}
"""

r = requests.put('http://localhost:9200/megacorp/employee/1?pretty', 
                 data = employee)

In [28]:
r = requests.get('http://localhost:9200/megacorp/_mappings?pretty')
print r.text

{
  "megacorp" : {
    "mappings" : {
      "employee" : {
        "properties" : {
          "about" : {
            "type" : "string"
          },
          "age" : {
            "type" : "long"
          },
          "first_name" : {
            "type" : "string"
          },
          "interests" : {
            "type" : "string"
          },
          "last_name" : {
            "type" : "string"
          }
        }
      }
    }
  }
}



In [29]:
r = requests.get('http://localhost:9200/tvseries/_mappings?pretty')
print r.text

{
  "tvseries" : {
    "mappings" : { }
  }
}



In [30]:
breaking_bad = requests.get('http://api.tvmaze.com/singlesearch/shows?q=breaking-bad')

breaking_bad.text


u'{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><em><strong>\\"Breaking Bad\\"</strong></em> follows protagonist Walter White, a chemistry teacher who lives in New Mexico with his wife and teenage son who has cerebral palsy. White is diagnosed with Stage III cancer and given a prognosis of two years left to live. With a new sense of fearlessness based on his medical prognosis

JSON formatter: https://jsonformatter.curiousconcept.com/

In [45]:
r = requests.put('http://localhost:9200/tvseries/')
r.text

u'{"acknowledged":true}'

In [53]:
r = requests.delete('http://localhost:9200/tvseries/')
r.text

u'{"acknowledged":true}'

In [54]:
breaking_bad.text

u'{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><em><strong>\\"Breaking Bad\\"</strong></em> follows protagonist Walter White, a chemistry teacher who lives in New Mexico with his wife and teenage son who has cerebral palsy. White is diagnosed with Stage III cancer and given a prognosis of two years left to live. With a new sense of fearlessness based on his medical prognosis

In [55]:
r = requests.post('http://localhost:9200/tvseries/serie/169', data = breaking_bad.text)
r.text

u'{"_index":"tvseries","_type":"serie","_id":"169","_version":1,"_shards":{"total":1,"successful":1,"failed":0},"created":true}'

In [56]:
r = requests.get('http://localhost:9200/tvseries/serie/169?pretty')
print r.text

{
  "_index" : "tvseries",
  "_type" : "serie",
  "_id" : "169",
  "_version" : 1,
  "found" : true,
  "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><em><strong>\"Breaking Bad\"</strong></em> follows protagonist Walter White, a chemistry teacher who lives in New Mexico with his wife and teenage son who has cerebral palsy. White is diagnosed with Stage III cancer and

In [57]:
r = requests.get('http://localhost:9200/tvseries/_search?q=genres:drama&pretty')
print r.text

{
  "took" : 9,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.15342641,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "169",
      "_score" : 0.15342641,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><em><strong>\"Break

In [61]:
series = ['blindspot','the knick','house of cards', 'orange is the new black',
          'true detective', 'game of thrones',
          'the tudors','isabel', 'versailles', 'los serrano']

for s in series:  
  data = requests.get('http://api.tvmaze.com/singlesearch/shows?q=' + s ) 
  id = data.json()['id']
  response = requests.post('http://localhost:9200/tvseries/serie/' + str(id), data = data)
  print s + " indexed: " + response.text 

blindspot indexed: {"_index":"tvseries","_type":"serie","_id":"1855","_version":3,"_shards":{"total":1,"successful":1,"failed":0},"created":false}
the knick indexed: {"_index":"tvseries","_type":"serie","_id":"51","_version":3,"_shards":{"total":1,"successful":1,"failed":0},"created":false}


house of cards indexed: {"_index":"tvseries","_type":"serie","_id":"175","_version":3,"_shards":{"total":1,"successful":1,"failed":0},"created":false}


orange is the new black indexed: {"_index":"tvseries","_type":"serie","_id":"170","_version":3,"_shards":{"total":1,"successful":1,"failed":0},"created":false}


true detective indexed: {"_index":"tvseries","_type":"serie","_id":"5","_version":3,"_shards":{"total":1,"successful":1,"failed":0},"created":false}


game of thrones indexed: {"_index":"tvseries","_type":"serie","_id":"82","_version":1,"_shards":{"total":1,"successful":1,"failed":0},"created":true}


the tudors indexed: {"_index":"tvseries","_type":"serie","_id":"712","_version":1,"_shards":{"total":1,"successful":1,"failed":0},"created":true}


isabel indexed: {"_index":"tvseries","_type":"serie","_id":"9274","_version":1,"_shards":{"total":1,"successful":1,"failed":0},"created":true}


versailles indexed: {"_index":"tvseries","_type":"serie","_id":"3386","_version":1,"_shards":{"total":1,"successful":1,"failed":0},"created":true}


los serrano indexed: {"_index":"tvseries","_type":"serie","_id":"6346","_version":1,"_shards":{"total":1,"successful":1,"failed":0},"created":true}


In [62]:
r = requests.get('http://localhost:9200/tvseries/_search?q=status:ended&pretty')
print r.text

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 1.7884574,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "169",
      "_score" : 1.7884574,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><em><strong>\"Breakin

## Tipos de datos (campos) :  Tipos básicos  

  * Texto: string *
  * Númericos:
      - byte, short, integer, long
      - float, double
  * Booleanos: boolean
  * Fechas: date / format
 

## Tipos de datos: Texto 
En realidad el tipo string se puede tratar de multiples formas: 
* Keywords: el campo no se indexa con un indice invertido

* Full Text: el campo se indexa con un índice invertido
      * Index
      * Analyze
      * Store

## Tipos de datos: Tipos complejos       
      * Null values
      * Arrays
      * Objects 
      * Nested
      * Tipos especificos ES 
        - Geocoordenadas - geo_point, geo_shape
        - IPs
        - Completion
  

## Analizadores

In [ ]:
### Whitespace Tokenizer 

[TODO] test a tokenizer
[TODO] Define a tokenizer 


In [17]:
summary = '''Breaking Bad follows protagonist Walter White, a chemistry teacher who lives in New Mexico with his wife 
          and teenage son who has cerebral palsy. White is diagnosed with Stage III cancer and given a prognosis of 
          two years left to live. With a new sense of fearlessness based on his medical prognosis, and a desire to secure
          his family's financial security, White chooses to enter a dangerous world of drugs and crime and ascends to power 
          in this world. The series explores how a fatal diagnosis such as White's releases a typical man from the daily 
          concerns and constraints of normal society and follows his transformation from mild family man to a kingpin 
          of the drug trade.'''

r = requests.post('http://localhost:9200/tvseries/_analyze?field=summary&pretty' , data = summary)
print r.text

{
  "tokens" : [ {
    "token" : "breaking",
    "start_offset" : 0,
    "end_offset" : 8,
    "type" : "<ALPHANUM>",
    "position" : 1
  }, {
    "token" : "bad",
    "start_offset" : 9,
    "end_offset" : 12,
    "type" : "<ALPHANUM>",
    "position" : 2
  }, {
    "token" : "follows",
    "start_offset" : 13,
    "end_offset" : 20,
    "type" : "<ALPHANUM>",
    "position" : 3
  }, {
    "token" : "protagonist",
    "start_offset" : 21,
    "end_offset" : 32,
    "type" : "<ALPHANUM>",
    "position" : 4
  }, {
    "token" : "walter",
    "start_offset" : 33,
    "end_offset" : 39,
    "type" : "<ALPHANUM>",
    "position" : 5
  }, {
    "token" : "white",
    "start_offset" : 40,
    "end_offset" : 45,
    "type" : "<ALPHANUM>",
    "position" : 6
  }, {
    "token" : "a",
    "start_offset" : 47,
    "end_offset" : 48,
    "type" : "<ALPHANUM>",
    "position" : 7
  }, {
    "token" : "chemistry",
    "start_offset" : 49,
    "end_offset" : 58,
    "type" : "<ALPHANUM>",
    "pos

In [23]:
name = '''Breaking Bad'''

r = requests.get('http://localhost:9200/tvseries/_analyze?field=name&pretty' , data = name)
print r.text

{
  "tokens" : [ {
    "token" : "breaking",
    "start_offset" : 0,
    "end_offset" : 8,
    "type" : "<ALPHANUM>",
    "position" : 1
  }, {
    "token" : "bad",
    "start_offset" : 9,
    "end_offset" : 12,
    "type" : "<ALPHANUM>",
    "position" : 2
  } ]
}



In [51]:
r = requests.get('http://localhost:9200/tvseries/_mappings?pretty')
print r.text

{
  "tvseries" : {
    "mappings" : {
      "tweet" : {
        "properties" : {
          "message" : {
            "type" : "string",
            "store" : true
          }
        }
      },
      "serie" : {
        "properties" : {
          "_links" : {
            "properties" : {
              "nextepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              },
              "previousepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              },
              "self" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              }
            }
          },
          "externals" : {
            "properties" : {
              "imdb" : {
                "type" : "string"
             

In [73]:
r = requests.delete('http://localhost:9200/tvseries/_mappings/tweet?pretty')
print r.text

{
  "acknowledged" : true
}



In [74]:
mapping = '''{
    "tweet" : {
        "properties" : {
            "message" : {"type" : "string", "store" : true, "analyzer": "keyword" }
        }
    }
}'''

In [75]:
r = requests.put('http://localhost:9200/tvseries/_mappings/tweet?pretty' , data = mapping)

print r.text

{
  "acknowledged" : true
}



Cuando tratamos de actualizar un mapping que ya existe nos ignora

In [55]:
mapping = '''{
    "serie" : {
        "properties" : {
            "message" : {"type" : "string", "store" : true }
        }
    }
}'''

r = requests.put('http://localhost:9200/tvseries/_mappings/serie?pretty' , data = mapping)

print r.text

{
  "acknowledged" : true
}



In [79]:
r = requests.get('http://localhost:9200/tvseries/_mappings?pretty')
print r.text

{
  "tvseries" : {
    "mappings" : {
      "tweet" : {
        "properties" : {
          "message" : {
            "type" : "string",
            "store" : true,
            "analyzer" : "keyword"
          }
        }
      },
      "serie" : {
        "properties" : {
          "_links" : {
            "properties" : {
              "nextepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              },
              "previousepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              },
              "self" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              }
            }
          },
          "externals" : {
            "properties" : {
              "imdb" : {
           

In [83]:
name = '''Breaking Bad'''

r = requests.get('http://localhost:9200/tvseries/_analyze?field=message&pretty' , data = name)
print r.text

{
  "tokens" : [ {
    "token" : "breaking",
    "start_offset" : 0,
    "end_offset" : 8,
    "type" : "<ALPHANUM>",
    "position" : 1
  }, {
    "token" : "bad",
    "start_offset" : 9,
    "end_offset" : 12,
    "type" : "<ALPHANUM>",
    "position" : 2
  } ]
}



In [87]:

        
r = requests.get('http://localhost:9200/tvseries/_analyze?analyzer=keyword&text=Breaking Bad&pretty' , data = name)
print r.text



{
  "tokens" : [ {
    "token" : "Breaking Bad",
    "start_offset" : 0,
    "end_offset" : 12,
    "type" : "word",
    "position" : 1
  } ]
}



In [88]:
r = requests.get('http://localhost:9200/tvseries/_analyze?analyzer=standard&text=Breaking Bad&pretty' , data = name)
print r.text


{
  "tokens" : [ {
    "token" : "breaking",
    "start_offset" : 0,
    "end_offset" : 8,
    "type" : "<ALPHANUM>",
    "position" : 1
  }, {
    "token" : "bad",
    "start_offset" : 9,
    "end_offset" : 12,
    "type" : "<ALPHANUM>",
    "position" : 2
  } ]
}



In [93]:
r = requests.get('http://localhost:9200/tvseries/_analyze?analyzer=stop&text=Breaking Bad&pretty' , data = name)
print r.text


{
  "tokens" : [ {
    "token" : "breaking",
    "start_offset" : 0,
    "end_offset" : 8,
    "type" : "word",
    "position" : 1
  }, {
    "token" : "bad",
    "start_offset" : 9,
    "end_offset" : 12,
    "type" : "word",
    "position" : 2
  } ]
}



In [95]:
r = requests.get('http://localhost:9200/tvseries/_analyze?analyzer=english&text=Breaking Bad&pretty' , data = name)
print r.text


{
  "tokens" : [ {
    "token" : "break",
    "start_offset" : 0,
    "end_offset" : 8,
    "type" : "<ALPHANUM>",
    "position" : 1
  }, {
    "token" : "bad",
    "start_offset" : 9,
    "end_offset" : 12,
    "type" : "<ALPHANUM>",
    "position" : 2
  } ]
}



## Estructura de un analizador

  - Filtro de caracteres (Character filters) -cero o más
    - Pasar a minúsculas 
    - Eliminar acentos y diacríticos
    - Eliminar signos de puntuación 
    - etc..
  - Tokenizador (Tokenizer) - uno 
  - Filtros de tokens (Token filters) -  cero o más  
    - Palabras de parada (Stopwords)
    - Lematizador (Stemming) 
    - Sinónimos 
    - Mapeo
    - etc.. 

## Tipos de analizadores

 * Standard analyzer
 * Simple analyzer
 * Whitespace analyzer
 * Language analyzer 


## Mappings

In [7]:
r = requests.get('http://localhost:9200/tvseries/_mappings?pretty')
print r.text

{
  "tvseries" : {
    "mappings" : {
      "serie" : {
        "properties" : {
          "_links" : {
            "properties" : {
              "nextepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              },
              "previousepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              },
              "self" : {
                "properties" : {
                  "href" : {
                    "type" : "string"
                  }
                }
              }
            }
          },
          "externals" : {
            "properties" : {
              "imdb" : {
                "type" : "string"
              },
              "thetvdb" : {
                "type" : "long"
              },
              "tvrage" : {
                "type" : "long"
              }


## Campo de búsqueda por defecto

## Múltiples mapeos para un campo

## Campos analizados vs no analizados

## Operadores de búsqueda - QueryDSL

 - Query context: “How well does this document match this query clause?” 
 - Filter contex: “Does this document match this query clause?” 
 


In [14]:
payload = """
{
  "query" : {
     "match_all" : { }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 7,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 15,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 1.0,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><em><strong>\"B

In [ ]:
### Query DSL - búsqueda de texto completo

In [15]:
import requests

payload = """
{
  "query" : {
     "query_string" : { "query" : "name:'Breaking Bad'" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 12,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.63418555,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.63418555,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p>

In [16]:
payload = """
{
    "query" : {
        "match" : {
            "summary" : "Mexico"
        }
    }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 8,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.09555809,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.09555809,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><

### QueryDSL - terms

In [22]:
payload = """
{
  "query" : {
     "term" : { "name": "breaking" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 7,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.76446474,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.76446474,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><

In [ ]:
Probar la misma query en mayusculas - no hay análisis asi que no matchea nada

https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-term-query.html

### Query DSL - terms

In [25]:
payload = """
{
  "query" : {
     "terms" : { "genres": ["drama", "crime"] }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 11,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 13,
    "max_score" : 0.8122232,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.8122232,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><e

Se pueden usar como query los terminos de otro documento - por ejemplo, los seguidores de un seguidor

In [28]:
payload = """
{
  "query" : {
    "range" : {
        "rating.average" : {
            "gte" : 9,
            "lte" : 10,
            "boost" : 2.0
        }
    }
}
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 8,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 1.0,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><em><strong>\"B

In [33]:
payload = """
{
  "query" : {
    "range" : {
        "premiered" : {
            "gte" : "now-1y/y",
            "lte" : "now/y"
        }
    }
}
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 8,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpOYIAbPRGM5XqNXQN",
      "_score" : 1.0,
      "_source":{"id":1855,"url":"http://www.tvmaze.com/shows/1855/blindspot","name":"Blindspot","type":"Scripted","language":"English","genres":["Action","Crime","Thriller","Mystery"],"status":"Running","runtime":60,"premiered":"2015-09-21","schedule":{"time":"22:00","days":["Monday"]},"rating":{"average":7.7},"weight":7,"network":{"id":1,"name":"NBC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":44628,"thetvdb":295647,"imdb":"tt4474344"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/28/70723.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/28/70723.jpg"},"summary":"<p>A b

### Query DSL - busquedas booleanas

In [91]:
payload = """
{
  "query" : {
    "bool" : {
        "must" : {
            "term" : { "user" : "kimchy" }
        },
        "must_not" : {
            "range" : {
                "age" : { "from" : 10, "to" : 20 }
            }
        },
        "should" : [
            {
                "term" : { "tag" : "wow" }
            },
            {
                "term" : { "tag" : "elasticsearch" }
            }
        ],
        "minimum_should_match" : 1,
        "boost" : 1.0
    }
  }
}
"""

payload = """
{
  "query" : {
    "bool" : {
        "must" : {
            "term" : { "genres" : "drama" }
        },
        "must_not" : {
            "term" : { "genres" : "comedia" }
        },
        "minimum_should_match" : 1,
        "boost" : 1.0
    }
  }
}
"""

payload = """
{
  "query" : {
    "bool" : {
        "must" : {
            "term" : { "genres" : "thriller" }
        },
        "must_not" : {
            "term" : { "summary" : "mexico" }
        },
        "minimum_should_match" : 1,
        "boost" : 1.0
    }
  }
}
"""

payload = """
{
  "query" : {
    "bool" : {
        "must" : {
            "term" : { "genres" : "thriller" }
        },
        "must_not" : {
            "term" : { "summary" : "mexico" }
        },
        "should" : {  
             "term" : { "genres" : "mystery" }
        },
        "minimum_should_match" : 1,
        "boost" : 1.0
    }
  }
}
"""




r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text


{
  "took" : 13,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 1.4099568,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpOYIAbPRGM5XqNXQN",
      "_score" : 1.4099568,
      "_source":{"id":1855,"url":"http://www.tvmaze.com/shows/1855/blindspot","name":"Blindspot","type":"Scripted","language":"English","genres":["Action","Crime","Thriller","Mystery"],"status":"Running","runtime":60,"premiered":"2015-09-21","schedule":{"time":"22:00","days":["Monday"]},"rating":{"average":7.7},"weight":7,"network":{"id":1,"name":"NBC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":44628,"thetvdb":295647,"imdb":"tt4474344"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/28/70723.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/28/70723.jpg"},"sum

### Query DSL - búsqueda  de frase 

In [99]:
payload = """
{
    "query" : {
        "match_phrase" : {
            "_all" : "Walter White"
        }
    }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty&explain', data = payload)
print r.text

{
  "took" : 81,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.15289295,
    "hits" : [ {
      "_shard" : 0,
      "_node" : "XsP6B7PiS22-RDItLiUi8A",
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.15289295,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com

### Query DSL - Búsqueda por matching parcial

In [42]:
payload = """
{
  "query" : {
    "prefix" : { "network.name" : "bb" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text


{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpOYavbPRGM5XqNXQO",
      "_score" : 1.0,
      "_source":{"id":11498,"url":"http://www.tvmaze.com/shows/11498/a-kick-up-the-eighties","name":"A Kick Up the Eighties","type":"Scripted","language":"English","genres":["Comedy"],"status":"Ended","runtime":30,"premiered":"1981-09-21","schedule":{"time":"","days":[]},"rating":{"average":null},"weight":0,"network":{"id":37,"name":"BBC Two","country":{"name":"United Kingdom","code":"GB","timezone":"Europe/London"}},"webChannel":null,"externals":{"tvrage":null,"thetvdb":195101,"imdb":null},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/38/96797.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/38/96797.jpg"},"summary":"<p><strong><em>\"A Kick Up

- wildcards 
- regexps

### Query DSL - busquedas borrosas

In [52]:
payload = """
{
  "query" : {
    "fuzzy" : { "summary" : "mejico" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text


{
  "took" : 67,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.09555809,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.09555809,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p>

In [72]:
payload = """
{
  "query" : {
    "fuzzy" : {
        "summary" : {
            "value" :         "mejico",
            "boost" :         1.0,
            "fuzziness" :     1,
            "prefix_length" : 0,
            "max_expansions": 100
        }
    }
    }
}"""


r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text


{
  "took" : 12,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.09555809,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.09555809,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p>

### Query DSL - Boosting

In [45]:

payload = """
{
  "query" : {
     "terms" : { 
         "genres": ["drama", "crime"],
         "boost" : 2.0
      }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
print r.text

{
  "took" : 11,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 13,
    "max_score" : 0.8122232,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.8122232,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p><e

### Búsqueda en multiples campos

### Búsqueda multiindice y multicampo

### Relevancia en Elasticsearch

### Relevancia:  *Practical Scoring Function*
  - recupera documentos usando un modelo booleano 
  - asigna la relevancia usando una formula basada en ideas 
    - TF-ID
    - Modelo de espacio vectorial
    

### Relevancia por defecto


$$ rel(q,d) = qNorm_q \cdot coord_{q,d} \cdot \sum_{t \in q}{tf_{t,d} \cdot idf_t^2 \cdot boost_t \cdot norm_{t,d}}$$

 - $qNorm_q$ : factor de normalización de las consultas - ignorar
 - $coord_{q,d}$ : *coordination factor* - sube la importancia de los documentos que tienen más terminos de la consulta 
 - $boost_t$: *query boost* - Sube la importancia de un determinado término
 - $norm_{t,d}$: Factor de normalizacion del indice - tiene en cuenta la longitud del documento y opcionalmente *index boost*

## Relevancia - Default score 
 
[TODO] Query sobre las series haciendo uso de summary



In [5]:
r = requests.get('http://localhost:9200/tvseries/_search?q=New Mexico&pretty')
print r.text

{
  "took" : 50,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.13050228,
    "hits" : [ {
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.13050228,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com/uploads/images/original_untouched/0/2400.jpg"},"summary":"<p>

### Explicando la relevancia 

In [ ]:
r = requests.get('http://localhost:9200/tvseries/_search?q=New Mexico&explain&pretty')

In [6]:
print r.text

{
  "took" : 65,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 11,
    "max_score" : 0.13050228,
    "hits" : [ {
      "_shard" : 0,
      "_node" : "ooHa5uc_QEOIos6c8iHMaw",
      "_index" : "tvseries",
      "_type" : "serie",
      "_id" : "AVKpN8mabPRGM5XqNXQE",
      "_score" : 0.13050228,
      "_source":{"id":169,"url":"http://www.tvmaze.com/shows/169/breaking-bad","name":"Breaking Bad","type":"Scripted","language":"English","genres":["Drama","Crime","Thriller"],"status":"Ended","runtime":60,"premiered":"2008-01-20","schedule":{"time":"22:00","days":["Sunday"]},"rating":{"average":9.3},"weight":2,"network":{"id":20,"name":"AMC","country":{"name":"United States","code":"US","timezone":"America/New_York"}},"webChannel":null,"externals":{"tvrage":18164,"thetvdb":81189,"imdb":"tt0903747"},"image":{"medium":"http://tvmazecdn.com/uploads/images/medium_portrait/0/2400.jpg","original":"http://tvmazecdn.com

[TODO] Explicar bien cada uno de los parámetros

## Modelos de relevancia alternativa (texto) 

Otras medidas de relavancia para documentos
  - Okapi BM 25 
  -Se puede elegir una funcion de similitud por campo. sin embargo requiere reindexar

Medidas de similitud entre cadenas:
  - Fuzzy similarity

[todo] ¿cómo cambiar la medida de relevancia?

## Otras medidas de relevancia (estructura) 

- We can take into account other relevance measures
     - Time - recency
     - Location - proximity
     - Other numerical fields
  - Difference with databases: algorithms are adapted to sort and get top k documents. 

## Definiendo la relevancia a medida

 - function score 
 - script score


[TODO] Ejemplo para tener una fecha o una puntuacion media en cuenta 

## Relevancia multicampo

## Búsqueda multicampo ?? 

Motivation: 

  * Different uses: 
    * Match different full text queries in different fields: title and author
    * Order and bool queries impact, boosting may also be used
    
    * Tuning: 
       * dis_max - selecting the score of the best fields
       * tie_breaker
       * multi_match - helper to direct the same query to different fields
       * we can select fields by using regular expressions 
       * cross fields entity search
       
   * best fields 
   * most fields 
   * cross fields 


## Integracion con la interfaz de búsqueda